In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.svm as svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [126]:
train_df = pd.read_csv('../EDA/scaled_data.csv')
train_df.drop(['Unnamed: 0'], axis=1, inplace=True)
train_df.head(1)

,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,직업,가족 구성원 수,산업군,나이,가입연수,도시구분,월간 수입
0,0,2,1,1,3,0,0,3,1,0,1,4,5,1,2,3,0


In [127]:
train_copy = train_df.copy()

In [ ]:
# 어느 산업군에서 어느 직업이 연체 여부가 높은지 확인

# 직업 수, 산업군 수의 편중을 완화하기 위하여 각 직업 별 연체 여부로 연체율 확인/ 각 산업군 별 연체 여부로 연체율 확인
# 위에서 확인한 각 산업군의 연체율을 총 평균연체율(10.71%)보다 높은 산업군 15개(총 33개의 직업 중)
# 각 산업군의 연체율을 총 평균연체율(10.71%)보다 높은 직업 8개(총 19개의 직업 중)

## 종사하는 산업군의 전체 인원수를 확인하여 너무 적은 경우 제외
## 직업에 종사하는 인원수를 확인하여 너무 적은 경우 제외

# 위의 사항 고려하여 top3 산업군과 top4 직업 선정
# 이를 결합하여 1위 산업군- 1위 직업, 2위 직업, 3위 직업 ... 3위 산업군- 1위 직업, 2위 직업, 3위 직업 비교
# 어느 파생변수가 가장 유의한지 -> 그 산업군에서 그 직업이 연체율이 높다고 예측 확인

In [ ]:
# 산업군 Top3 : 8(레스토랑-0.195%), 0(건설업-0.155%), 24(자영업-0.137%)
# 직업 Top4 : 15(저임금 노동자-0.237%), 12(운전자-0.150%), 17(보안업계종사자-0.149%), 5(단순노동자-0.140%)


In [ ]:
train_copy.head()

In [128]:
# 산업군_직업 컬럼 생성 함수
def create_job_column(df, ind_value, job_value):

#    새로운 컬럼을 생성하고 특정 조건에 따라 값을 할당하는 함수

#    Parameters:
#    - df: 데이터프레임
#    - job_value: 직업 구분코드
#    - ind_value : 산업군 구분코드

#    Returns:
#    - df: 새로운 컬럼을 추가한 데이터프레임
    
  new_column_name = f"{ind_value}_{job_value}"
  # 조건에 맞는 행에 1을 할당하여 새로운 컬럼에 추가
  df[new_column_name] = (df['산업군'] == ind_value) & (df['직업'] == job_value)
  # True/False를 1/0으로 변환
  df[new_column_name] = df[new_column_name].astype(int)
    
  return df
# 함수 사용 예시
# create_job_column(your_df, ind_value, job_value)

In [131]:
create_job_column(train_copy, 8, 15)
create_job_column(train_copy, 8, 12)
create_job_column(train_copy, 8, 17)
create_job_column(train_copy, 8, 5)

,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,...,가족 구성원 수,산업군,나이,가입연수,도시구분,월간 수입,8_17,8_15,8_12,8_5
0,0,2,1,1,3,0,0,3,1,0,...,4,5,1,2,3,0,0,0,0,0
1,0,1,1,0,1,1,0,3,1,0,...,2,16,2,1,1,3,0,0,0,0
2,0,2,0,1,1,0,0,3,1,0,...,2,16,1,1,1,1,0,0,0,0
3,0,2,1,0,2,0,0,3,1,0,...,3,17,1,1,3,2,0,0,0,0
4,0,2,0,1,1,0,0,3,1,0,...,2,16,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59984,0,2,0,1,3,0,1,3,1,0,...,1,5,3,2,1,0,0,0,0,0
59985,1,1,1,1,1,0,0,3,1,0,...,2,24,2,0,1,3,0,0,0,0
59986,1,2,0,0,1,1,1,3,1,0,...,1,22,0,1,1,2,0,0,0,0
59987,0,2,1,1,1,0,3,3,1,0,...,1,19,2,1,1,1,0,0,0,0


In [134]:
(train_copy['8_17']==1).value_counts()

8_17
False    59942
True        47
Name: count, dtype: int64

In [ ]:
# 파생변수 '가족크기범주' 생성
train_copy['가족크기범주'] = pd.cut(train_copy['가족 구성원 수'], bins=[0, 2, 4,float('inf')],  
                    labels=['0','1', '2'])

In [ ]:
# 데이터타입 category -> int 변경 
train_copy['가족크기범주'] = train_copy['가족크기범주'].dropna().astype(int)

In [ ]:
# 파생변수 'combinedFY' 생성
# 조건 설정
conditions = [
(train_copy['가족크기범주'] == 0) & (train_copy['가입연수'] == 0),
(train_copy['가족크기범주'] == 0) & (train_copy['가입연수'] == 1),
(train_copy['가족크기범주'] == 0) & (train_copy['가입연수'] == 2),
(train_copy['가족크기범주'] == 0) & (train_copy['가입연수'] == 3),

(train_copy['가족크기범주'] == 1) & (train_copy['가입연수'] == 0),
(train_copy['가족크기범주'] == 1) & (train_copy['가입연수'] == 1),
(train_copy['가족크기범주'] == 1) & (train_copy['가입연수'] == 2),
(train_copy['가족크기범주'] == 1) & (train_copy['가입연수'] == 3),


(train_copy['가족크기범주'] == 2) & (train_copy['가입연수'] == 0),
(train_copy['가족크기범주'] == 2) & (train_copy['가입연수'] == 1),
(train_copy['가족크기범주'] == 2) & (train_copy['가입연수'] == 2),
(train_copy['가족크기범주'] == 2) & (train_copy['가입연수'] == 3),
]
# 할당할 값 설정
values = [0, 1, 2, 3, 4, 5, 6, 7, 8,9,10,11]

# np.select를 사용하여 조건에 맞는 값을 'y' 컬럼에 할당
train_copy['combinedfY'] = np.select(conditions, values, default=0)
#train_copy = train_copy.drop(['차량 소유 여부', '부동산 소유 여부', '주거 형태', '근속연수'], axis=1)
train_copy.head(2)

In [ ]:
# 파생변수 'age_income' 생성
# 임금수준 & 나이활용한 파생변수
conditions1 = [
(train_copy['나이'] == 0) & (train_copy['월간 수입'] == 0),
(train_copy['나이'] == 0) & (train_copy['월간 수입'] == 1),
(train_copy['나이'] == 0) & (train_copy['월간 수입'] == 2),
(train_copy['나이'] == 0) & (train_copy['월간 수입'] == 3),

(train_copy['나이'] == 1) & (train_copy['월간 수입'] == 0),
(train_copy['나이'] == 1) & (train_copy['월간 수입'] == 1),
(train_copy['나이'] == 1) & (train_copy['월간 수입'] == 2),
(train_copy['나이'] == 1) & (train_copy['월간 수입'] == 3),

(train_copy['나이'] == 2) & (train_copy['월간 수입'] == 0),
(train_copy['나이'] == 2) & (train_copy['월간 수입'] == 1),
(train_copy['나이'] == 2) & (train_copy['월간 수입'] == 2),
(train_copy['나이'] == 2) & (train_copy['월간 수입'] == 3),

(train_copy['나이'] == 3) & (train_copy['월간 수입'] == 0),
(train_copy['나이'] == 3) & (train_copy['월간 수입'] == 1),
(train_copy['나이'] == 3) & (train_copy['월간 수입'] == 2),
(train_copy['나이'] == 3) & (train_copy['월간 수입'] == 3),

(train_copy['나이'] == 4) & (train_copy['월간 수입'] == 0),
(train_copy['나이'] == 4) & (train_copy['월간 수입'] == 1),
(train_copy['나이'] == 4) & (train_copy['월간 수입'] == 2),
(train_copy['나이'] == 4) & (train_copy['월간 수입'] == 3),

]
# 할당할 값 설정
values = [0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
# np.select를 사용하여 조건에 맞는 값을 'y' 컬럼에 할당
train_copy['age_income'] = np.select(conditions1, values, default=0)
train_copy.head(2)

In [ ]:
# 카이제곱 검정은 파생변수가 범주형일 때 유의미한지 검증하는 대표적인 지표.
# 결과로 나온 출력에서 p-value가 0.05 미만이면서 chi2가 가장 높은 파생변수가
# 카이제곱 검정으로 봤을때 가장 의미가 있는 피쳐.

# 일단 가설을 설정했다면 그 가설에 필요한 변수만 빼서 카이제곱 검정을 돌려. 
# 그래서 0.05이상 나오면 그 가설과 파생변수는 유의미하니 사용 가능.
# 그리고 이것도 통계적 요소 대립가설을 언급하며 논리를 펼칠 것.

In [135]:
# p-value값 확인
from scipy.stats import chi2_contingency
chi2_results = {}
categorical_features = ['8_15', '8_12', '8_17', '8_5']

# 나의 가설은 이렇다.
# 비교적 나이가 젊은 층 중 기혼자가 미혼자에 비해 재정 관리 능력이 더 뛰어날 것.
# 특히 월간 수입이 이러한 범위에 있는 사람은 연체 확률이 더 높거나 낮을 것.
# 그럼 내 가설에 필요한 피처들을 카이제곱 검정에 넣어.

In [136]:
# p-value값 확인
for feature in categorical_features:
    contingency_table = pd.crosstab(train_copy[feature], train_copy['TARGET'])
    
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    chi2_results[feature] = {'chi2': chi2, 'p-value': p}


chi2_results

{'8_15': {'chi2': 0.11114709784753601, 'p-value': 0.7388419416561753},
 '8_12': {'chi2': 6.278177989877958e-09, 'p-value': 0.9999367796533193},
 '8_17': {'chi2': 1.3521077297086408, 'p-value': 0.2449099784118336},
 '8_5': {'chi2': 2.1830295476658343, 'p-value': 0.13953954646675246}}

In [ ]:
train_copy

In [137]:
# feature와 label 분리
feature = train_copy.drop('TARGET', axis=1)
label = train_copy['TARGET']

In [138]:
# train test 분리
X_train_old, X_test, y_train_old , y_test = train_test_split(feature, label, test_size=0.3 , random_state=42)

In [139]:
y_train_old.value_counts()

TARGET
0    37465
1     4527
Name: count, dtype: int64

In [140]:
from imblearn.over_sampling import SMOTE
X_train, y_train = SMOTE(random_state = 22).fit_resample(X_train_old, y_train_old)

In [141]:
y_train.value_counts()

TARGET
0    37465
1    37465
Name: count, dtype: int64

In [142]:
# 1. Decision Tree를 이용한 학습
# 1-1. GridSearchCV를 적용해 Decision Tree의 교차검증 및 하이퍼파라미터 튜닝

dt = DecisionTreeClassifier()

parameters = {'max_depth' : [2, 3, 4, 5],
             'min_samples_split' : [1, 3, 5, 7, 9]}

grid_dt = GridSearchCV(dt, param_grid = parameters, cv=3, refit=True)
grid_dt.fit(X_train, y_train)

dt = grid_dt.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_dt.best_params_}")
print(f"최고 예측 정확도: {grid_dt.best_score_:.4f}")

최적 하이퍼 파라미터: {'max_depth': 5, 'min_samples_split': 3}
최고 예측 정확도: 0.6843


In [143]:
# 모델로부터 예측 확률 계산
# predicted_probabilities = dt.predict_proba(X_test)

# 임계값 설정
# threshold = 0.6  # 임계값 설정 (예시로 0.6으로 설정)

# 예측값 변환
# predicted_classes = (predicted_probabilities[:, 1] >= threshold).astype(int)  # 양성 클래스의 확률에 대해 임계값 적용


In [144]:
# 임계값 재설정
from sklearn.preprocessing import Binarizer

custom_threshold = 0.5

pred_proba_1 = pred_proba[:,1].reshape(-1,1)

binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_1)
custom_predict = binarizer.transform(pred_proba_1)
custom_predict
get_clf_eval(y_test, custom_predict)

NameError: name 'get_clf_eval' is not defined

In [145]:
# Decision Tree 적용
pred = dt.predict(X_test)
pred_proba = dt.predict_proba(X_test)
pred_proba_1 = pred_proba[:, 1]

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test , pred)
rec = recall_score(y_test , pred)
f1score = f1_score(y_test, pred)
auc_score = roc_auc_score(y_test , pred_proba_1)
print(f'의사결정나무 정확도 : {acc:.3f}')
print(f'의사결정나무 정밀도 : {prec:.3f}')
print(f'의사결정나무 재현율 : {rec:.3f}')
print(f'의사결정나무 f1_score : {f1score:.3f}')
print(f'의사결정나무 roc_auc : {auc_score:.3f}')

의사결정나무 정확도 : 0.699
의사결정나무 정밀도 : 0.166
의사결정나무 재현율 : 0.461
의사결정나무 f1_score : 0.244
의사결정나무 roc_auc : 0.614


In [146]:
# 트리 기반 모델의 특성 중요도 확인
for feature, importance in zip(X_train.columns, dt.feature_importances_):
    print(f"{feature}: {importance}")

성별: 0.17520632186909546
차량 소유 여부: 0.17426409606578339
부동산 소유 여부: 0.0012887755684592112
수입 유형: 0.07569731229146599
최종 학력: 0.25027056746682547
결혼 여부: 0.003930252698453974
주거 형태: 0.0
휴대전화 소유 여부: 0.0
이메일 소유 여부: 0.0
직업: 0.0
가족 구성원 수: 0.012843883883974897
산업군: 0.00206681853335694
나이: 0.24148157227928582
가입연수: 0.062216268196943966
도시구분: 0.0
월간 수입: 0.0007341311463548633
8_17: 0.0
8_15: 0.0
8_12: 0.0
8_5: 0.0


In [147]:
# 2. RandomForest를 이용한 학습
# 2-1. GridSearchCV를 적용해 RandomForest의 교차검증 및 하이퍼파라미터 튜닝

rf = RandomForestClassifier()

parameters = {
    'max_depth' : [6, 8, 12],
    'min_samples_split' : [16, 24]
}

grid_rf = GridSearchCV(rf, param_grid = parameters, cv=3, refit=True)
grid_rf.fit(X_train, y_train)

rf = grid_rf.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_rf.best_params_}")
print(f"최고 예측 정확도: {grid_rf.best_score_:.4f}")

최적 하이퍼 파라미터: {'max_depth': 12, 'min_samples_split': 16}
최고 예측 정확도: 0.7588


In [148]:
# RandomForest 적용

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
pred_proba = rf.predict_proba(X_test)
pred_proba_1 = pred_proba[:, 1]

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test , pred)
rec = recall_score(y_test , pred)
f1score = f1_score(y_test, pred)
auc_score = roc_auc_score(y_test , pred_proba_1)

print(f'랜덤포레스트 정확도 : {acc:.3f}')
print(f'랜덤포레스트 정밀도 : {prec:.3f}')
print(f'랜덤포레스트 재현율 : {rec:.3f}')
print(f'랜덤포레스트 f1_score : {f1score:.3f}')
print(f'랜덤포레스트 roc_auc : {auc_score:.3f}')

랜덤포레스트 정확도 : 0.783
랜덤포레스트 정밀도 : 0.141
랜덤포레스트 재현율 : 0.207
랜덤포레스트 f1_score : 0.168
랜덤포레스트 roc_auc : 0.580


In [149]:
# 트리 기반 모델의 특성 중요도 확인
for feature, importance in zip(X_train.columns, rf.feature_importances_):
    print(f"{feature}: {importance}")

성별: 0.04360537433202821
차량 소유 여부: 0.04204020762253011
부동산 소유 여부: 0.029372847159125623
수입 유형: 0.054367897115122694
최종 학력: 0.055854642751264
결혼 여부: 0.0580027111248939
주거 형태: 0.023167617127706035
휴대전화 소유 여부: 2.15185623429432e-06
이메일 소유 여부: 0.0134872584205425
직업: 0.13971801362314362
가족 구성원 수: 0.0717148249205003
산업군: 0.14957877479543277
나이: 0.09055480832981798
가입연수: 0.07118084811377204
도시구분: 0.06630924101930775
월간 수입: 0.09062948288151718
8_17: 0.0001788932524258211
8_15: 3.076045368938274e-05
8_12: 7.08512697491871e-05
8_5: 0.00013279383119657235


In [150]:
# 3. Logistic regrssion 을 이용한 학습
# 3-1. GridSearchCV를 적용해 Logistic regrssion의 교차검증 및 하이퍼파라미터 튜닝

lr = LogisticRegression()

parameters = {'penalty': ['l2','l1'],
          'C':[0.01,0.1,1,10]}

grid_lr = GridSearchCV(lr, param_grid = parameters, cv=3, refit=True)
grid_lr.fit(X_train, y_train)

lr = grid_lr.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_lr.best_params_}")
print(f"최고 예측 정확도: {grid_lr.best_score_:.4f}")

최적 하이퍼 파라미터: {'C': 10, 'penalty': 'l2'}
최고 예측 정확도: 0.7009


In [151]:
#3-2. Logistic Regression 적용
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
pred_proba = lr.predict_proba(X_test)
pred_proba_1 = pred_proba[:, 1]

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test , pred)
rec = recall_score(y_test , pred)
auc_score = roc_auc_score(y_test , pred_proba_1)
f1score = f1_score(y_test, pred)

print(f'Logistic regrssion 정확도 : {acc:.3f}')
print(f'Logistic regrssion 정밀도 : {prec:.3f}')
print(f'Logistic regrssion 재현율 : {rec:.3f}')
print(f'Logistic regrssion f1_score : {f1score:.3f}')
print(f'Logistic regrssion roc_auc : {auc_score:.3f}')

Logistic regrssion 정확도 : 0.675
Logistic regrssion 정밀도 : 0.152
Logistic regrssion 재현율 : 0.455
Logistic regrssion f1_score : 0.228
Logistic regrssion roc_auc : 0.604


In [152]:
# 학습된 모델의 특성 중요도 확인
feature_importance = lr.coef_[0]  # 특성의 가중치 또는 중요도

# 특성별 중요도 출력
for feature, importance in zip(X_train.columns, feature_importance):
    print(f"{feature}: {importance}")

성별: -1.0469855203625182
차량 소유 여부: -1.0901345877361188
부동산 소유 여부: -0.315163410534757
수입 유형: -0.15697812636794936
최종 학력: -0.7530915668461345
결혼 여부: -0.07773754312119277
주거 형태: 0.02897957669113229
휴대전화 소유 여부: 2.499841456144387
이메일 소유 여부: -1.3351252989849174
직업: -0.017023064725541288
가족 구성원 수: -0.32792040131633304
산업군: -0.006770419883082787
나이: -0.38040544822586214
가입연수: -0.511115248028163
도시구분: -0.11736181846916659
월간 수입: -0.24983919590749493
8_17: -0.01858445596946226
8_15: -0.003732340834815636
8_12: -0.01649623215277584
8_5: -0.0006029733222051346


In [153]:
# 5. KNN 을 이용한 학습
# 5-1. GridSearchCV를 적용해 KNN의 교차검증 및 하이퍼파라미터 튜닝

knn = KNeighborsClassifier()

parameters = {'n_neighbors': [3, 5, 7, 9],
              'weights': ['uniform', 'distance']
          }

grid_knn = GridSearchCV(knn, param_grid = parameters, cv=3, refit=True)
grid_knn.fit(X_train, y_train)

knn = grid_knn.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_knn.best_params_}")
print(f"최고 예측 정확도: {grid_knn.best_score_:.4f}")

In [ ]:
# 5-2 knn 적용
knn = KNeighborsClassifier()

knn.fit(X_train, y_train)
pred = knn.predict(X_test) 

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test , pred)
rec = recall_score(y_test , pred)
auc_score = roc_auc_score(y_test , pred)
f1score = f1_score(y_test, pred)

print(f'KNN 정확도 : {acc:.3f}')
print(f'KNN 정밀도 : {prec:.3f}')
print(f'KNN 재현율 : {rec:.3f}')
print(f'KNN f1_score : {f1score:.3f}')
print(f'KNN roc_auc : {auc_score:.3f}')

In [1]:
# 6. xgboost 를 이용한 학습
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators = 150,
                            learning_rate = 0.2,
                            max_depth = 10,
                            min_child_weight = 5,
                            gamma = 10)

# fit
xgb_model.fit(X_train, y_train)

# 예측
y_pred = xgb_model.predict(X_test)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, y_pred)

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test , pred)
rec = recall_score(y_test , pred)
auc_score = roc_auc_score(y_test , pred)
f1score = f1_score(y_test, pred)


# score print
print(f'xgboost 정확도 : {acc:.3f}')
print(f'xgboost 정밀도 : {prec:.3f}')
print(f'xgboost 재현율 : {rec:.3f}')
print(f'xgboost f1_score : {f1score:.3f}')
print(f'xgboost roc_auc : {auc_score:.3f}')

NameError: name 'X_train' is not defined

In [ ]:
# 5. Lightgbm을 이용한 학습
# 5-1. GridSearchCV를 적용해 Lightgbm의 교차검증 및 하이퍼파라미터 튜닝

from lightgbm import LGBMClassifier
lgbm= LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False, force_row_wise=True)
evals = [(X_test, y_test)]
lgbm.fit(X_train, y_train, eval_metric='logloss', eval_set=evals)

lgbm.fit(X_train, y_train)
pred = lgbm.predict(X_test)
pred_proba = lgbm.predict_proba(X_test)[:,1]

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test, pred)
rec = recall_score(y_test , pred)
auc_score = roc_auc_score(y_test , pred)
f1score = f1_score(y_test, pred)

print(f'lightgbm 정확도 : {acc:.3f}')
print(f'lightgbm 정밀도 : {prec:.3f}')
print(f'lightgbm 재현율 : {rec:.3f}')
print(f'lightgbm f1_score : {f1score:.3f}')
print(f'lightgbm roc_auc : {auc_score:.3f}')

In [ ]:
# 산업군_직업, 산업군_직업2를 같이 돌리면 성능 떨어짐
# 산업군_직업만 추가해서 돌리는게 더 나음